# Climate change impact factors 

Climate change impact factor calculation is easier, since regionality does not have to be taken into account. Thus the calculation simplifies to taking the consumption based impact factors from exiobase and multiplying them by characterisation factors of different ecosystems (terrestrial and aquatic) from lc-impact.

NOTE: Climate change impact factor calculation method is not clear. The paper states: "In terms of the biodiversity impacts of climate change, we take into account carbon dioxide, methane, fossil methane and nitrous oxide. We chose impact factors that take all effects into account for a period of 100 years for both terrestrial and aquatic ecosystems(42). With the spatial component missing from the climate change biodiversity impact analyses, we then multiplied the biodiversity impact factor of each gas with its respective counterpart factor in EXIOBASE."
However, in Table S5 "Climate change midpoint | ILCD recommended CF | Global warming potential 100 years" (found from impacts folder) is defined as the stressore name in exiobase. It's not clear what 'respective counterpart factor in EXIOBASE' refers to exactly.

Exiobase categories:
- (Check groupings from the code below)

LC-Impact stressors
- Climate change (Terrestrial ecosystems (PDF/C))

In [12]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [13]:
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

In [14]:
# load and prepare lc-impact data
lci = pd.read_excel(arguments["lc_impact_path"] + "/2-climate change/Climate change CFs.xlsx", sheet_name=6, header=[0,1])
# take only sixth column
lci = lci.iloc[:4, [0, 6, 10]]
lci.columns = [' '.join(col).strip() for col in lci.columns]
lci.rename(columns={lci.columns[0]: "Substance", lci.columns[1]: "All effects 100yrs (terrestrial)", lci.columns[2]: "All effects 100yrs (aquatic)"}, inplace=True)
lci

,Substance,All effects 100yrs (terrestrial),All effects 100yrs (aquatic)
0,Carbon dioxide,1.759831e-15,5.469745e-16
1,Methane,4.927527e-14,1.531529e-14
2,Fossil methane,5.279493e-14,1.640923e-14
3,Nitrous oxide,4.663552e-13,1.449482e-13


## Calculate climate change impact factors
1. Aggregate relevant stressors from exiobase (correct stressors unclear)
2. Multiply the aggregated row with either terrestrial or aquatic factor of the given category
3. Combine the factors of different stressors into two categories (aquatic and terrestrial)

In [13]:
# TODO: this grouping should be checked
groups = exio3_19.satellite.get_index(as_dict=True, grouping_pattern={
    "CO2.*": "CO2 - Total",
    "CH4 - waste - air": "CH4",
    "CH4 - agriculture - air": "CH4",
    "CH4 - non combustion - Oil refinery - air": "CH4 fossil",
    "CH4 - non combustion - Mining of sub-bituminous coal - air": "CH4 fossil",
    "CH4 - non combustion - Mining of lignite (brown coal) - air": "CH4 fossil",
    "CH4 - non combustion - Mining of antracite - air": "CH4 fossil",
    "CH4 - non combustion - Mining of coking coal - air": "CH4 fossil",
    "CH4 - non combustion - Mining of bituminous coal - air": "CH4 fossil",
    "CH4 - non combustion - Extraction/production of (natural) gas - air": "CH4 fossil",
    "CH4 - non combustion - Extraction/production of crude oil": "CH4 fossil", 
    "CH4 - combustion - air": "CH4 fossil",
    "NOx.*": "NOx - Total",
    })

exio3_19.satellite_agg = exio3_19.satellite.copy(new_name="Aggregated CO2 accounts")

for df_name, df in zip(exio3_19.satellite_agg.get_DataFrame(data=False, with_unit=True, with_population=False),
exio3_19.satellite_agg.get_DataFrame(data=True, with_unit=True, with_population=False)):
    if df_name == "unit":
        exio3_19.satellite_agg.__dict__[df_name] = df.groupby(groups).apply(lambda x: " & ".join(x.unit.unique()))
    else:
        exio3_19.satellite_agg.__dict__[df_name] = df.groupby(groups).sum()

In [14]:
exio3_19.satellite_agg.M.loc['CO2 - Total']

region  sector                                           
AT      Paddy rice                                                0.000000
        Wheat                                                831730.023474
        Cereal grains nec                                    393237.244108
        Vegetables, fruit, nuts                              152094.453476
        Oil seeds                                            405632.138550
                                                                 ...      
WM      Membership organisation services n.e.c. (91)         371941.611917
        Recreational, cultural and sporting services (92)    368443.008485
        Other services (93)                                  463759.404543
        Private households with employed persons (95)        195273.005683
        Extra-territorial organizations and bodies                0.000000
Name: CO2 - Total, Length: 9800, dtype: float64

In [60]:
exio3_19.satellite_agg.M.loc['CH4']

region  sector                                           
AT      Paddy rice                                              0.000000
        Wheat                                                 238.174076
        Cereal grains nec                                     195.946451
        Vegetables, fruit, nuts                               178.093943
        Oil seeds                                             155.410456
                                                                ...     
WM      Membership organisation services n.e.c. (91)         1270.246314
        Recreational, cultural and sporting services (92)    3585.110573
        Other services (93)                                  5019.349385
        Private households with employed persons (95)        1226.385321
        Extra-territorial organizations and bodies              0.000000
Name: CH4, Length: 9800, dtype: float64

In [61]:
exio3_19.satellite_agg.M.loc['CH4 fossil']

region  sector                                           
AT      Paddy rice                                              0.000000
        Wheat                                                1178.043340
        Cereal grains nec                                     962.402000
        Vegetables, fruit, nuts                               358.712271
        Oil seeds                                             509.837701
                                                                ...     
WM      Membership organisation services n.e.c. (91)          508.177642
        Recreational, cultural and sporting services (92)     579.070489
        Other services (93)                                   850.785437
        Private households with employed persons (95)         382.501080
        Extra-territorial organizations and bodies              0.000000
Name: CH4 fossil, Length: 9800, dtype: float64

In [62]:
exio3_19.satellite_agg.M.loc['NOx - Total']

region  sector                                           
AT      Paddy rice                                              0.000000
        Wheat                                                5481.644986
        Cereal grains nec                                    2989.027420
        Vegetables, fruit, nuts                               709.596784
        Oil seeds                                            3388.733276
                                                                ...     
WM      Membership organisation services n.e.c. (91)         1172.429865
        Recreational, cultural and sporting services (92)    1173.160911
        Other services (93)                                  1376.118654
        Private households with employed persons (95)         716.157791
        Extra-territorial organizations and bodies              0.000000
Name: NOx - Total, Length: 9800, dtype: float64

In [ ]:
# calculate aquatic factors
aquatic = exio3_19.satellite_agg.M.loc['CO2 - Total'] * lci['All effects 100yrs (aquatic)'].values[0] + \
           exio3_19.satellite_agg.M.loc['CH4'] * lci['All effects 100yrs (aquatic)'].values[1] + \
           exio3_19.satellite_agg.M.loc['CH4 fossil'] * lci['All effects 100yrs (aquatic)'].values[2] + \
           exio3_19.satellite_agg.M.loc['NOx - Total'] * lci['All effects 100yrs (aquatic)'].values[3]
aquatic

region  sector                                           
AT      Paddy rice                                           0.000000e+00
        Wheat                                                1.272468e-09
        Cereal grains nec                                    6.671383e-10
        Vegetables, fruit, nuts                              1.946603e-10
        Oil seeds                                            7.238076e-10
                                                                 ...     
WM      Membership organisation services n.e.c. (91)         4.011772e-10
        Recreational, cultural and sporting services (92)    4.359856e-10
        Other services (93)                                  5.439641e-10
        Private households with employed persons (95)        2.356742e-10
        Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [68]:
# calculate terrestrial factors
terrestrial = exio3_19.satellite_agg.M.loc['CO2 - Total'] * lci['All effects 100yrs (terrestrial)'].values[0] + \
           exio3_19.satellite_agg.M.loc['CH4'] * lci['All effects 100yrs (terrestrial)'].values[1] + \
           exio3_19.satellite_agg.M.loc['CH4 fossil'] * lci['All effects 100yrs (terrestrial)'].values[2] + \
           exio3_19.satellite_agg.M.loc['NOx - Total'] * lci['All effects 100yrs (terrestrial)'].values[3]
terrestrial

region  sector                                           
AT      Paddy rice                                           0.000000e+00
        Wheat                                                4.094029e-09
        Cereal grains nec                                    2.146445e-09
        Vegetables, fruit, nuts                              6.262985e-10
        Oil seeds                                            2.328772e-09
                                                                 ...     
WM      Membership organisation services n.e.c. (91)         1.290744e-09
        Recreational, cultural and sporting services (92)    1.402736e-09
        Other services (93)                                  1.750145e-09
        Private households with employed persons (95)        7.582560e-10
        Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [69]:
pd.DataFrame(aquatic).to_csv("csv/pdf-climate-aquatic.csv", index=True)
pd.DataFrame(terrestrial).to_csv("csv/pdf-climate-terrestrial.csv", index=True)